<a href="https://colab.research.google.com/github/oscarnavmac/Proyecto-Final-de-Redes-Convolucionales/blob/main/Proyecto_Final_Oscar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Knowledge Distillation of Convolutional Neural Networks

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Dataset

In [3]:
# Preprocesamiento de datos para CIFAR-10.
transforms_cifar = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Cargando los datos
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms_cifar)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms_cifar)

100%|██████████| 170498071/170498071 [00:13<00:00, 12941430.57it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
# Dataloaders: usaremos un tamaño de lote de 128
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2)

## Model and utility functions

In [5]:
# Class for the teacher model
class DeepNN(nn.Module):
    def __init__(self, num_classes=10):
        super(DeepNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [14]:
# Class for the teacher model
class LightNN(nn.Module):
    def __init__(self, num_classes=10):
        super(LightNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [7]:
def train(model, train_loader, epochs, learning_rate, device):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    model.train()

    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            # inputs: Una coleccion de imagenes con batch_size tamano de lote
            # labels: Un vector de dimensionalidad batch_size con enteros denotando la clase para cada imagen
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader)}")

In [8]:
def test (model, test_loader, device):
    model.to(device)
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels, = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    return accuracy

In [10]:
torch.manual_seed(42)
nn_deep = DeepNN(num_classes=10).to(device)
train(nn_deep, train_loader, epochs=10, learning_rate=0.001, device=device)
test_accuracy_deep = test(nn_deep, test_loader, device)

Epoch 1/10, Loss: 1.3446038572684578
Epoch 2/10, Loss: 0.871539844271472
Epoch 3/10, Loss: 0.6765290761695189
Epoch 4/10, Loss: 0.5307565338318915
Epoch 5/10, Loss: 0.40622283476392934
Epoch 6/10, Loss: 0.293585380210596
Epoch 7/10, Loss: 0.21417938470078246
Epoch 8/10, Loss: 0.15814204624546763
Epoch 9/10, Loss: 0.13053985068674587
Epoch 10/10, Loss: 0.12362362659724473
Test Accuracy: 75.25%


In [15]:
# Instantiate the lightweight network:
torch.manual_seed(42)
nn_light = LightNN(num_classes=10).to(device)
train(nn_light, train_loader, epochs=10, learning_rate=0.001, device=device)
test_accuracy_light_ce = test(nn_light, test_loader, device)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch 1/10, Loss: 1.4697696412615764
Epoch 2/10, Loss: 1.1571572598289042
Epoch 3/10, Loss: 1.0252658753748745
Epoch 4/10, Loss: 0.9236393098331168
Epoch 5/10, Loss: 0.8497249261497537
Epoch 6/10, Loss: 0.7811627134947521
Epoch 7/10, Loss: 0.7166911406285318
Epoch 8/10, Loss: 0.6580311824446139
Epoch 9/10, Loss: 0.6077518277156079
Epoch 10/10, Loss: 0.556332155261808
Test Accuracy: 70.46%


## Knowledge Distillation

In [22]:
def train_knowledge_distillation(teacher, student, train_loader, epochs, learning_rate, T,
soft_target_loss_weight, ce_loss_weight, device):

    ce_loss = nn.CrossEntropyLoss()
    optimizer = optim.Adam(student.parameters(), lr=learning_rate)

    teacher.eval()
    student.eval()

    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            with torch.no_grad():
                teacher_logits = teacher(inputs)

            student_logits = student(inputs)

            soft_targets = nn.functional.softmax(teacher_logits / T, dim=-1)
            soft_prob = nn.functional.log_softmax(student_logits / T, dim=-1)

            soft_target_loss = torch.sum(soft_targets * (soft_targets.log() - soft_prob)) / soft_prob.size()[0] * (T**2)

            label_loss = ce_loss(student_logits, labels)

            loss = soft_target_loss_weight * soft_target_loss + ce_loss_weight * label_loss

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"{epoch+1}/{epochs}, Loss: {running_loss / len(train_loader)}")

In [23]:
torch.manual_seed(42)
new_nn_light = LightNN(num_classes=10).to(device)

train_knowledge_distillation(teacher=nn_deep, student=new_nn_light, train_loader=train_loader,
                             epochs=10, learning_rate=0.001, T=2, soft_target_loss_weight=0.25,
                             ce_loss_weight=0.75, device=device)

test_accuracy_light_ce_and_kd = test(new_nn_light, test_loader, device)

1/10, Loss: 2.3772521247644254
2/10, Loss: 1.864064818148113
3/10, Loss: 1.6360787796547345
4/10, Loss: 1.4699511579845264
5/10, Loss: 1.3420475236595135
6/10, Loss: 1.2309744679714407
7/10, Loss: 1.120672263452769
8/10, Loss: 1.0134513981811835
9/10, Loss: 0.9202194927293627
10/10, Loss: 0.8374271621484586
Test Accuracy: 70.64%


In [24]:
print(f"Teacher accuracy: {test_accuracy_deep:.2f}%")
print(f"Student accuracy without teacher: {test_accuracy_light_ce:.2f}%")
print(f"Student accuracy with CE + KD: {test_accuracy_light_ce_and_kd:.2f}%")

Teacher accuracy: 75.25%
Student accuracy without teacher: 70.46%
Student accuracy with CE + KD: 70.64%


In [25]:
total_params_deep = "{:,}".format(sum(p.numel() for p in nn_deep.parameters()))
print(f"DeepNN parameters: {total_params_deep}")
total_params_light = "{:,}".format(sum(p.numel() for p in nn_light.parameters()))
print(f"LightNN parameters: {total_params_light}")

DeepNN parameters: 1,186,986
LightNN parameters: 267,738
